# The SQL Queries to answer the questions the questions of the project

## 1. Which 10 wines does the company need to highlight to increase the sales?

In [1]:
#First, we need to access the database.
import sqlite3

connexion = sqlite3.connect("vivino.db")
cursor = connexion.cursor()

In [7]:
# To answer the question, we order by ratings average. 
cursor.execute("""
    SELECT wines.name AS name, regions.name AS region, countries.name AS country, ratings_average AS rating
    FROM wines
    INNER JOIN regions ON regions.id = wines.region_id
    INNER JOIN countries ON countries.code = regions.country_code
    ORDER BY rating DESC
    LIMIT 10;
""")
result = cursor.fetchall()

print("We need to highlight these hight-rated wines: ")
for i in range(len(result)):
    print(i+1, ". " + result[i][0]+ " from " + result[i][1] + ", " + result[i][2] + ", rated ", result[i][3], ".")

We need to highlight these hight-rated wines: 
1 . Amarone della Valpolicella Classico Riserva from Amarone della Valpolicella Classico, Italie, rated  4.8 .
2 . Cabernet Sauvignon from Rutherford, États-Unis, rated  4.8 .
3 . Mágico from Rioja, Espagne, rated  4.8 .
4 . Fratini Bolgheri Superiore from Bolgheri Superiore, Italie, rated  4.8 .
5 . Cristal Rosé Vinothèque from Champagne, France, rated  4.8 .
6 . Batard-Montrachet Grand Cru from Bâtard-Montrachet Grand Cru, France, rated  4.8 .
7 . IX Estate Red from Napa Valley, États-Unis, rated  4.7 .
8 . Special Selection Cabernet Sauvignon from Napa Valley, États-Unis, rated  4.7 .
9 . Unico Reserva Especial Edición from Ribera del Duero, Espagne, rated  4.7 .
10 . Unico from Ribera del Duero, Espagne, rated  4.7 .


## 2. Which wine exporting country should the company prioritise due to the limited marketing budget for this year?

In [8]:
# To answer this question, this query
query2 = """SELECT
    country_name,
    natural_wines_count,
    non_natural_wines_count,
    natural_acidic_wines_count,
    natural_sweet_wines_count,
    natural_tannic_wines_count,
    non_natural_acidic_wines_count,
    non_natural_sweet_wines_count,
    non_natural_tannic_wines_count
FROM (
    SELECT
        c.name AS country_name,
        SUM(CASE WHEN w.is_natural = 1 THEN 1 ELSE 0 END) AS natural_wines_count,
        SUM(CASE WHEN w.is_natural = 0 THEN 1 ELSE 0 END) AS non_natural_wines_count,
        SUM(CASE WHEN w.is_natural = 1 AND w.acidity > w.sweetness AND w.acidity > w.tannin THEN 1 ELSE 0 END) AS natural_acidic_wines_count,
        SUM(CASE WHEN w.is_natural = 1 AND w.sweetness > w.acidity AND w.sweetness > w.tannin THEN 1 ELSE 0 END) AS natural_sweet_wines_count,
        SUM(CASE WHEN w.is_natural = 1 AND w.tannin > w.acidity AND w.tannin > w.sweetness THEN 1 ELSE 0 END) AS natural_tannic_wines_count,
        SUM(CASE WHEN w.is_natural = 0 AND w.acidity > w.sweetness AND w.acidity > w.tannin THEN 1 ELSE 0 END) AS non_natural_acidic_wines_count,
        SUM(CASE WHEN w.is_natural = 0 AND w.sweetness > w.acidity AND w.sweetness > w.tannin THEN 1 ELSE 0 END) AS non_natural_sweet_wines_count,
        SUM(CASE WHEN w.is_natural = 0 AND w.tannin > w.acidity AND w.tannin > w.sweetness THEN 1 ELSE 0 END) AS non_natural_tannic_wines_count
    FROM
        countries c
    LEFT JOIN
        regions r ON c.code = r.country_code
    LEFT JOIN
        wines w ON r.id = w.region_id
    GROUP BY
        c.name
    ORDER BY non_natural_wines_count DESC
) AS subquery;"""

cursor.execute(query2)
results = cursor.fetchall()

# Print column headers
print("{:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20}".format(
    "Country", "Natural Wines", "Non-Natural Wines",
    "Natural Acidic", "Natural Sweet", "Natural Tannic",
    "Non-Natural Acidic", "Non-Natural Sweet", "Non-Natural Tannic"
))

# Print each row
for row in results:
    country_name, natural_wines_count, non_natural_wines_count, \
    natural_acidic_wines_count, natural_sweet_wines_count, natural_tannic_wines_count, \
    non_natural_acidic_wines_count, non_natural_sweet_wines_count, non_natural_tannic_wines_count = row
    print("{:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20} {:<20}".format(
        country_name, natural_wines_count, non_natural_wines_count,
        natural_acidic_wines_count, natural_sweet_wines_count, natural_tannic_wines_count,
        non_natural_acidic_wines_count, non_natural_sweet_wines_count, non_natural_tannic_wines_count
    ))

Country              Natural Wines        Non-Natural Wines    Natural Acidic       Natural Sweet        Natural Tannic       Non-Natural Acidic   Non-Natural Sweet    Non-Natural Tannic  
Italie               18                   365                  12                   1                    2                    201                  67                   62                  
France               13                   318                  10                   0                    0                    121                  0                    53                  
Espagne              2                    92                   2                    0                    0                    39                   0                    34                  
États-Unis           0                    74                   0                    0                    0                    41                   0                    26                  
Portugal             0                    28           

## 3. Which wineries should we award and why?

In [9]:
#To answer this question, we display the wines that have #1 ranking 2 years in row, and the highest rating given by most people
#The wineries are anonymized for subjectifity!

query3 = """
SELECT DISTINCT wines.name, wines.ratings_average, wines.ratings_count, wines.winery_id
FROM wines
JOIN vintages ON wines.id = vintages.wine_id
JOIN vintage_toplists_rankings ON vintages.id = vintage_toplists_rankings.vintage_id
JOIN toplists ON vintage_toplists_rankings.top_list_id = toplists.id
WHERE vintage_toplists_rankings.rank = 1 AND vintage_toplists_rankings.previous_rank = 1
GROUP BY toplists.name
ORDER BY wines.ratings_average DESC, wines.ratings_count DESC
LIMIT 3
"""
cursor.execute(query3)
result = cursor.fetchall()

for i in range(len(result)):

    print(f"The {str(i+1)} degree award goes to winery number {str(result[i][3])}, for their {result[i][0]}, rated {str(result[i][1])} by {str(result[i][2])} users!")

The 1 degree award goes to winery number 14919, for their Cabernet Sauvignon, rated 4.8 by 2941 users!
The 2 degree award goes to winery number 82979, for their Toscana, rated 4.7 by 16284 users!
The 3 degree award goes to winery number 3286, for their Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne, rated 4.7 by 9564 users!


## 4. What wines can be described by the following taste: *coffee, toast, green apple, cream, citrus*? 

In [10]:
#importing the libriries to make table output
from prettytable import PrettyTable
from collections import defaultdict

# To answer the question, we looked up the keywords and corresponding wines

query4 = '''
SELECT 
    w.name AS WineName,
    k.name AS Keyword,
    SUM(kw.count) AS KeywordCount
FROM 
    wines w
JOIN 
    keywords_wine kw ON w.id = kw.wine_id
JOIN 
    keywords k ON kw.keyword_id = k.id
WHERE 
    k.name IN ('coffee', 'toast', 'green apple', 'cream', 'citrus')
GROUP BY 
    w.name, k.name
HAVING 
    SUM(kw.count) >= 10

    '''
cursor.execute(query4)
results = cursor.fetchall()

# Create a PrettyTable object with the appropriate headers
table = PrettyTable()
table.field_names = ["Wine Name", "Citrus",
                     "Toast", "Coffee", "Cream", "Green Apple"]

# Process the results into a dictionary
wines = defaultdict(lambda: defaultdict(int))
for wine_name, keyword, count in results:
    wines[wine_name][keyword] = count

# Fill the table with wine data
for wine_name, keywords in wines.items():
    if len(keywords) == 5:  # Check if the wine has all 5 keywords
        # Append rows to the table with the count for each keyword
        table.add_row([
            wine_name,
            keywords.get('citrus', 0),
            keywords.get('toast', 0),
            keywords.get('coffee', 0),
            keywords.get('cream', 0),
            keywords.get('green apple', 0)
        ])

# Sort the table by a column if desired (e.g., 'Wine Name')
table.sortby = "Wine Name"

# Print the table
print(table)

+----------------------------------------------------+--------+-------+--------+-------+-------------+
|                     Wine Name                      | Citrus | Toast | Coffee | Cream | Green Apple |
+----------------------------------------------------+--------+-------+--------+-------+-------------+
|            Belle Epoque Brut Champagne             |  423   |  478  |   28   |  207  |     224     |
|               Blanc des Millénaires                |  181   |  320  |   54   |  149  |      66     |
|                   Brut Champagne                   |  2596  |  4330 |  290   |  1564 |     893     |
|                Brut Rosé Champagne                 |  763   |  918  |   18   |  740  |     130     |
|             Clos des Goisses Champagne             |   78   |  106  |   18   |   65  |      24     |
|                   Clos du Mesnil                   |   62   |   66  |   18   |   16  |      19     |
|        Comtes de Champagne Blanc de Blancs         |  720   |  1174 |  

## 5. What are the top 3 most common grapes all over the world, and what are the 5 best-rated wines for each type?

In [22]:
# Find the top 3 most common grapes all over the world
query5 = """
    SELECT grapes.name, grape_id, wines_count AS count
    FROM most_used_grapes_per_country
    INNER JOIN grapes ON grapes.id = most_used_grapes_per_country.grape_id
    GROUP BY grapes.name
    ORDER BY count DESC
    LIMIT 3;
"""
cursor.execute(query5)
result = cursor.fetchall()
top_grape = []
for i in range(len(result)):
    top_grape.append(result[i][0])
    print(f"The number {i+1} popular grape is {result[i][0]}. There are {result[i][2]} wines made of it!")

The number 1 popular grape is Cabernet Sauvignon. There are 801751 wines made of it!
The number 2 popular grape is Chardonnay. There are 604208 wines made of it!
The number 3 popular grape is Pinot Noir. There are 572334 wines made of it!


In [36]:
# for each most common top 3 grape, give us the the 5 best rated wines.
results = []
for grape in top_grape:
    query = f"""
    SELECT 
            wines.name AS name, 
            regions.name AS region, 
            countries.name AS country, 
            ratings_average AS rating
        FROM wines
        INNER JOIN regions ON regions.id = wines.region_id
        INNER JOIN countries ON countries.code = regions.country_code
        WHERE wines.name LIKE '%{grape}%'
        ORDER BY rating DESC, ratings_count DESC
        LIMIT 5;
    """
    cursor.execute(query)
    result = cursor.fetchall()
    results.append(result)

for i, grape_results in enumerate(results):
    print(f"The best wines from {top_grape[i]} are: ")
    for wine in grape_results:
        print(f"{wine[0]}, from {wine[1]}, {wine[2]}, rated {wine[3]}")
    print("----------------------------------------")


The best wines from Cabernet Sauvignon are: 
Cabernet Sauvignon, from Rutherford, États-Unis, rated 4.8
Special Selection Cabernet Sauvignon, from Napa Valley, États-Unis, rated 4.7
Cabernet Sauvignon, from Napa Valley, États-Unis, rated 4.6
Cabernet Sauvignon (Signature), from Napa Valley, États-Unis, rated 4.6
CASK 23 Cabernet Sauvignon, from Napa Valley, États-Unis, rated 4.6
----------------------------------------
The best wines from Chardonnay are: 
Chloe Chardonnay, from Russian River Valley, États-Unis, rated 4.5
Reserve Chardonnay, from Greyton, Afrique du Sud, rated 4.5
Bentrock Vineyard Chardonnay, from Sta. Rita Hills, États-Unis, rated 4.5
One Sixteen Chardonnay, from Russian River Valley, États-Unis, rated 4.4
Chardonnay, from Western Cape, Afrique du Sud, rated 4.4
----------------------------------------
The best wines from Pinot Noir are: 
Grace Vineyard Pinot Noir, from Dundee Hills, États-Unis, rated 4.5
Finn Pinot Noir, from Russian River Valley, États-Unis, rated 4

## 6. What is a leaderboard for the average wine and vintage rating in each country?


In [41]:
#Leaderboard for wines
query6 = '''
    SELECT 
        c.name AS country,
        AVG(w.ratings_average) AS avg_rating
    FROM 
        countries c
    JOIN 
        regions r ON c.code = r.country_code
    JOIN 
        wines w ON r.id = w.region_id
    GROUP BY 
        c.name
    ORDER BY 
        AVG(w.ratings_average) DESC;
'''

cursor.execute(query6)
country_results = cursor.fetchall()

# Create a table
from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ["Country", "Average Wine Rating"]

for country, avg_rating in country_results:
    table.add_row([country, f"{avg_rating:.2f}"])

print(table)



+----------------+---------------------+
|    Country     | Average Wine Rating |
+----------------+---------------------+
|   Allemagne    |         4.50        |
|     Israël     |         4.50        |
|   États-Unis   |         4.49        |
|    Moldavie    |         4.48        |
|    Hongrie     |         4.47        |
| Afrique du Sud |         4.46        |
|   Australie    |         4.46        |
|     France     |         4.45        |
|    Espagne     |         4.44        |
|    Portugal    |         4.44        |
|     Chili      |         4.43        |
|     Italie     |         4.43        |
|   Argentine    |         4.42        |
|    Roumanie    |         4.40        |
|     Grèce      |         4.40        |
|     Suisse     |         4.35        |
|    Croatie     |         4.30        |
+----------------+---------------------+


In [42]:
#Leaderbard for vintages
vintage_query = '''
    SELECT 
        c.name AS country,
        AVG(v.ratings_average) AS avg_rating
    FROM 
        countries c
    JOIN 
        regions r ON c.code = r.country_code
    JOIN 
        wines w ON r.id = w.region_id
    JOIN
        vintages v ON w.id = v.wine_id
    GROUP BY 
        c.name
    ORDER BY 
        AVG(v.ratings_average) DESC;
'''


cursor.execute(vintage_query)
vintage_results = cursor.fetchall()

table = PrettyTable()
table.field_names = ["Country", "Average Vintage Rating"]

for country, avg_rating in vintage_results:  # Use sorted data
    table.add_row([country, f"{avg_rating:.2f}"])

print(table)


+----------------+------------------------+
|    Country     | Average Vintage Rating |
+----------------+------------------------+
|    Roumanie    |          4.50          |
|    Croatie     |          4.40          |
|   Argentine    |          4.32          |
|     Chili      |          4.12          |
|    Portugal    |          3.93          |
|    Moldavie    |          3.87          |
|   États-Unis   |          3.70          |
| Afrique du Sud |          3.66          |
|     Italie     |          3.63          |
|    Espagne     |          3.38          |
|    Hongrie     |          3.17          |
|     France     |          3.07          |
|   Australie    |          2.83          |
|     Grèce      |          2.30          |
|     Suisse     |          2.20          |
|   Allemagne    |          1.70          |
|     Israël     |          0.00          |
+----------------+------------------------+


## 7. Which wines should the company recommend to a VIP client who likes Cabernet Sauvignon?

In [13]:
query7 = """
    SELECT wines.name
    FROM wines
    JOIN regions ON wines.region_id = regions.id
    JOIN countries ON regions.country_code = countries.code
    JOIN most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
    JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
    WHERE grapes.name = 'Cabernet Sauvignon'
    GROUP BY wines.name
    ORDER BY 
        ((wines.ratings_average * wines.ratings_count) + 
        ((
            SELECT SUM(wines.ratings_average) 
            FROM wines
        ) / (
            SELECT COUNT(DISTINCT id) 
            FROM wines
        ))
    ) / (wines.ratings_count + 1) DESC
    LIMIT 5;
"""
cursor.execute(query7)
results = cursor.fetchall()
print("The Cabernet Sauvignon admirers will also like:")
for i in range(len(results)):
    print(i+1, results[i][0])

The Cabernet Sauvignon admirers will also like:
1 Mágico
2 Unico
3 Special Selection Cabernet Sauvignon
4 Grange
5 Unico Reserva Especial Edición
